<style>
.rendered_html {
    font-size: 17px;
    font-family: Times New Roman, serif;
    text-align:justify}
</style>

# Using Bio-Signals to Predict Smoker Status

I chose the following dataset: https://www.kaggle.com/datasets/gauravduttakiit/smoker-status-prediction-using-biosignals

I decided on using this dataset because it provides over 55,000 rows of biological data, including bio-signals like the levels of individual hormones and proteins, about both smokers and non-smokers. I am interested in determining if there are any strong trends or clusters between any combination of these bio-signals and smoking. 